In [32]:
import pandas as pd
import numpy as np
import math

Например, мы можем представить те же самые фильмы как набор из трёх вещественных чисел:

фильм $A = (1.1, 2.3, 5.1)$;   
фильм $B = (1.3, 2.1, 4.9)$;   
фильм $C = (5.1, 6.2, 1.1)$.

Чтобы вычислить косинусную близость, нам понадобится следующая формула:

![](https://lms.skillfactory.ru/assets/courseware/v1/19ae75390814099a1aaf8a660888b0d6/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/DST_MATH_ML_15_2_5.png)

### Задание 2.1
Вычислите косинусную близость между векторами А и С. Результат округлите до трёх знаков после точки-разделителя.

In [35]:
A = np.array([1.1,2.3,5.1])
C = np.array([5.1,6.2,1.1])

ans = A@C/(np.linalg.norm(A)*np.linalg.norm(C))
round(ans,3)

0.551

Итак, чтобы построить рекомендательную систему на основе контента, необходимо:

* Для каждого продукта создать характеризующие его признаки.
* Найти показатель близости между всеми продуктами.
* Порекомендовать пользователю продукты, которые показывают наибольшую близость с теми продуктами, которые он высоко оценил.    
    
Давайте реализуем подобную рекомендательную систему на практике. Будем работать с 
[датасетом](https://lms.skillfactory.ru/assets/courseware/v1/747dae7bf99b18ce3b24bd34aa7bc29b/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/netflix_titles.zip), содержащим информацию об оценивании фильмов на платформе ***Netflix***.

Признаки в данных
* show_id — id фильма,
* type — его тип (фильм или сериал),
* title — название,
* director — режиссер,
* cast — актерский состав,
* country — страна,
* date_added — дата добавления,
* release_year — год выхода на экраны,
* rating — рейтинг,
* duration — продолжительность,
* listened_in — жанр(-ы),
* description — описание.

In [36]:
df = pd.read_csv('./data/netflix_titles.csv')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,"August 14, 2020",2020,TV-MA,4 Seasons,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s2,Movie,7:19,Jorge Michel Grau,"Demián Bichir, Héctor Bonilla, Oscar Serrano, ...",Mexico,"December 23, 2016",2016,TV-MA,93 min,"Dramas, International Movies",After a devastating earthquake hits Mexico Cit...
2,s3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,"December 20, 2018",2011,R,78 min,"Horror Movies, International Movies","When an army recruit is found dead, his fellow..."
3,s4,Movie,9,Shane Acker,"Elijah Wood, John C. Reilly, Jennifer Connelly...",United States,"November 16, 2017",2009,PG-13,80 min,"Action & Adventure, Independent Movies, Sci-Fi...","In a postapocalyptic world, rag-doll robots hi..."
4,s5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,"January 1, 2020",2008,PG-13,123 min,Dramas,A brilliant group of students become card-coun...


В первую очередь нам необходимо определить, на основании чего мы будем рассматривать близость фильмов. Выберем для этой задачи описание фильма, ведь в нём, скорее всего, содержится много информации. Однако описание — это текст. Есть много подходов к преобразованию текста в вектор, и мы будем использовать подход **TF-IDF** (Term Frequency-Inverse Document Frequency).
>Показатель **TD-IDF** — это индикатор того, насколько релевантно слово в контексте документа.

Его можно определить следующим образом:
$$\text{TF-IDF(слова) = TF(слова) * IDF (слова)}$$
$\text{TF слова} = \frac{\text{Количество раз, когда слово встретилось в тексте}}{\text{Количество всех слов в тексте}}$   
$\text{IDF слова} = log \left (\frac{\text{Общее кол-во документов}}{\text{Кол-во документов, в которых встречается слово}}\right )$   
    
Этот показатель возрастает пропорционально количеству раз, когда слово встречается в тексте, и уменьшается пропорционально количеству слов во всех текстах в целом.

Таким образом:

* Коэффициент будет выше, если слово характерно именно для этого текста, то есть встречается в данном тексте часто, но не встречается в других текстах.
* Коэффициент будет ниже, если слово не встречается почти нигде или встречается одинаковое количество раз во всех текстах, то есть не характеризует никакой текст в отдельности.

In [37]:
# Чтобы преобразовать текст по этому принципу, нам понадобится соответствующая функция 
# из библиотеки sklearn — импортируем её:
from sklearn.feature_extraction.text import TfidfVectorizer

# Далее учтём стоп-слова, т. е. предлоги и другие служебные части речи, которые не несут 
# содержательной информации, и с учётом этого определим нашу модель:
model = TfidfVectorizer(stop_words='english')

# Заполним пропуски пустыми строками:
df['description'] = df['description'].fillna('')

# Трансформируем наши описания в матрицу:
feature_matrix = model.fit_transform(df['description'])

### Задание 2.2
Сколько столбцов в получившейся матрице?

In [38]:
feature_matrix.shape[1]

17905

Теперь необходимо вычислить косинусную близость. Можно сделать это так:
```python
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)
```
>Обратите внимание! Мы используем здесь linear_kernel(), а не cosine_similarity(), так как в косинусном расстоянии в знаменателе реализуется нормировка векторов, а **TF-IDF** создаёт уже нормализованные векторы.

Вернём индексацию и уберём дубликаты из данных:
```python
indices = pd.Series(df.index,index=df['title']).drop_duplicates()
```
Теперь пропишем функцию для создания рекомендаций:
```python
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]
```
Например, если мы хотим найти рекомендации по фильму "Star Trek", то функция будет выдавать следующий результат:

<div style="text-align: left;">
<img src="https://lms.skillfactory.ru/assets/courseware/v1/bd77bbe01c25390b5e2d2aa3a3731fad/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/DST_MATH_ML_15_2_6.png"  width="450" height="450" align="left">
</div>

In [39]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(feature_matrix, feature_matrix)

In [40]:
indices = pd.Series(df.index,index=df['title']).drop_duplicates()

In [41]:
def get_recommendations(title):
    idx = indices[title]
    #вычисляем попарные коэффициенты косинусной близости
    scores = list(enumerate(cosine_sim[idx]))
    #сортируем фильмы на основании коэффициентов косинусной близости по убыванию
    scores = sorted(scores, key=lambda x: x[1], reverse=True)
    #выбираем десять наибольших значений косинусной близости; нулевую не берём, т. к. это тот же фильм
    scores =   scores[1:11]
    #забираем индексы
    ind_movie = [i[0] for i in scores]
    #возвращаем названия по индексам
    return df['title'].iloc[ind_movie]

In [42]:
get_recommendations("Star Trek")

5788             Star Trek: The Next Generation
5787                      Star Trek: Enterprise
5786                 Star Trek: Deep Space Nine
5557                     She's Out of My League
134                                  7 Days Out
6664                        The Midnight Gospel
6023                                     Teresa
4863    Pinkfong & Baby Shark's Space Adventure
5104                                       Rats
5970                             Tales by Light
Name: title, dtype: object

### Задание 2.3
Найдите вторую рекомендацию для детского фильма "Balto", вышедшего на экраны в 1995 году:


In [43]:
df[df.title == "Balto"].show_id

708    s709
Name: show_id, dtype: object

In [44]:
get_recommendations("Balto").iloc[1]

'Vroomiz'

## 3. Коллаборативная фильтрация
Мы рассмотрели несколько вариантов коллаборативной фильтрации на простейших примерах, и теперь пришло время практики с настоящими данными. Сначала мы будем использовать подход memory-based в модификации item-based, а затем SVD. В результате применения обоих алгоритмов мы сможем сравнить получившееся качество.   
В нашей задаче мы будем использовать датасет [movielens](https://lms.skillfactory.ru/assets/courseware/v1/6e47046882bad158b0efbb84cd5cb987/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/u.data.txt), который содержит информацию о фильмах и выставленных рейтингах с сайта https://movielens.org/.

In [45]:
# Импортируем необходимые нам компоненты и считаем данные с помощью специального метода Reader:

from surprise import Dataset
from surprise import Reader
from surprise.dataset import BUILTIN_DATASETS # с помощью данного объекта мы можем использовать 
                                              # встроенные датасеты

data = Dataset.load_from_file(
    "./data/u.data.txt",
    reader=Reader(line_format="user item rating timestamp", sep="\t"),
)

Чтобы обучать рекомендательные системы с помощью `surprise`, мы создали объект `Dataset`. Объект `surprise.dataset` — это набор данных, который содержит следующие поля в указанном порядке:
* идентификаторы пользователей,
* идентификаторы элементов,
* соответствующая оценка.   
Преобразуем данные к формату `pandas.DataFrame` для удобной работы с ними:

In [46]:
df = pd.DataFrame(data.raw_ratings, columns=['userId', 'movieId', 'rating', 'timestamp'])

В данных присутствуют следующие признаки:
* userId — идентификаторы пользователей сайта movielens;
* movieId — идентификаторы фильмов;
* rating — оценки фильмов, выставленные пользователями по шкале от 1 до 5;
* timestamp — время оценки фильма пользователем. Данный формат представления времени показывает, сколько секунд прошло с 1 января 1970 года.

### Задание 3.1
Сколько уникальных фильмов в наборе данных?

In [47]:
df.movieId.nunique()

1682

### Задание 3.2
Сколько уникальных пользователей в наборе данных?

In [48]:
df.userId.nunique()

943

### Задание 3.3
Какая оценка встречается в наборе данных чаще всего? Введите ответ в виде целого числа.

In [49]:
df.rating.mode()[0]

4.0

Библиотека surprise очень похожа на библиотеку sklearn, и тоже позволяет разбить данные на обучающую и тестовую выборки всего одной функцией — `train_test_split()`.

### Задание 3.4
Разбейте данные на обучающую и тестовую выборки. Объём тестовой выборки должен составлять 25 % от общего объёма данных. В качестве значения параметра `random_state` возьмите число `13`.

Сколько объектов попало в тестовую выборку?

In [50]:
from surprise.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.25,random_state=13)
len(test)

25000

Импортируем функции для построения рекомендательных систем (`SVD` — для **model-based**-подхода и `KNNBasic` — для **memory-basic**-подхода) и для оценки качества результата.

In [51]:
from surprise import SVD, KNNBasic, accuracy

Теперь реализуем обычную коллаборативную фильтрацию. Выберем оценку схожести через **косинусную близость** и **item-based**-подход:

In [52]:
sim_options = {
    'name': 'cosine',
    'user_based': False}
 
knn = KNNBasic(sim_options=sim_options)

Обучим алгоритм:

In [53]:
knn.fit(train)

Computing the cosine similarity matrix...
Done computing similarity matrix.


Теперь давайте посмотрим, какие рекомендации мы получили, с помощью следующей программы:

In [54]:
predictions = knn.test(test)
predictions[:5]

[Prediction(uid='7', iid='633', r_ui=5.0, est=4.199452349030111, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='422', iid='287', r_ui=3.0, est=3.4703437660463736, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='804', iid='163', r_ui=3.0, est=3.5716736533692854, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='189', iid='480', r_ui=5.0, est=4.222825780855538, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='238', iid='546', r_ui=3.0, est=3.473417286928204, details={'actual_k': 17, 'was_impossible': False})]

Информация о каждой паре будет содержать следующие характеристики:

* uid — id пользователя;
* iid — id элемента;
* r_ui (float) — реальный рейтинг, который этот пользователь поставил этому элементу;
* est (float) — предсказанный рейтинг.* 

### Задание 3.5
1. Каков реальный рейтинг, выставленный пользователем с **ID** ***500*** для фильма с **ID** ***699***?

In [55]:
[x.r_ui for x in predictions if (x.uid,x.iid) ==('500','699')][0]

3.0

2. Каков прогнозируемый рейтинг для пользователя с **ID** ***500*** и фильма с **ID** ***699***? Ответ округлите до двух знаков после точки-разделителя.

In [56]:
ans = [x.est for x in predictions if (x.uid,x.iid) ==('500','699')][0]
round(ans,2)

3.47

Теперь необходимо вычислить RMSE для получившихся предсказаний:

In [57]:
accuracy.rmse(predictions)

RMSE: 1.0272


1.0271678039029761

Если округлить результат до сотых, получаем 1.03.
>Итак, мы построили систему рекомендаций и даже оценили её качество. Но как же вывести рекомендации для конкретного пользователя?

Для начала давайте оформим наши предсказания в таблицу и отсортируем их по прогнозируемому рейтингу:

In [58]:
pred = pd.DataFrame(predictions)
pred.sort_values(by=['est'],inplace=True,ascending = False)

Теперь мы можем вывести рекомендуемые для конкретного пользователя фильмы, начиная от наиболее релевантного (с точки зрения рекомендаций) и заканчивая наименее релевантным.

In [59]:
recom = pred[pred.uid =='849']['iid'].to_list()
recom

['234', '427', '568', '174']

### Задание 3.6
Реализуйте **user-based**-алгоритм. Какое значение **RMSE** получилось для коллаборативной фильтрации типа **user-based**? Ответ округлите до двух знаков после точки-разделителя.

In [60]:
sim_options = {
    'name': 'cosine',
    'user_based': True}
 
knn = KNNBasic(sim_options=sim_options)
knn.fit(train)
predictions = knn.test(test)
ans = accuracy.rmse(predictions)
round(ans,2)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0175


1.02

### Задание 3.7
Теперь давайте сравним полученные результаты с результатами **SVD**-алгоритма. Реализуйте **SVD** с параметрами по умолчанию.

Какое значение **RMSE** получилось для **SVD**? Ответ округлите до двух знаков после точки-разделителя.

In [61]:
svd = SVD()

svd.fit(train)
predictions = svd.test(test)
ans = accuracy.rmse(predictions)
round(ans,2)

RMSE: 0.9414


0.94

## 4. Гибридные модели

In [62]:
from lightfm import LightFM
from lightfm.cross_validation import random_train_test_split
from lightfm.evaluation import precision_at_k, recall_at_k

C:\Users\Manych\anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Работать мы будем с датасетом [goodreads_book](https://lms.skillfactory.ru/assets/courseware/v1/c977535583bf2f85a2d15617e672d8f4/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/Gooddreadbooks.zip).

***Goodreads*** — это сайт, на котором люди могут добавлять книги в каталоги, искать их, изучать аннотации и отзывы. Пользователи также могут создавать сообщества, в которых они рекомендуют друг другу различную литературу, ведут блоги и устраивают обсуждения.

Подгрузим все файлы, относящиеся к этому набору данных:

In [63]:
ratings = pd.read_csv('./data/ratings.csv') #поставленные оценки
books = pd.read_csv('./data/books.csv') #информация о книгах
tags = pd.read_csv('./data/tags.csv') #информация о тегах
book_tags = pd.read_csv('./data/book_tags.csv') #книги с тегами 

Сначала посмотрим на набор данных **books**: в этих данных есть обычный **id** книги, а есть **id** книги в системе **Goodreads** — этот **id** отображён в признаке `goodreads_book_id`. В других данных (**book_tags**) указан только **id** книги в системе **Goodreads**, поэтому нам необходимо добавить туда обычный **id**.

### Задание 4.1
Добавьте в набор данных **book_tags** признак с обычным **id** книги, используя соответствие обычного **id** и **id** в системе **Goodreads**.

Какой обычный **id** у книги, которая имеет **id** 5 в системе **Goodreads**?

In [64]:
# book_tags = book_tags.merge(books[["book_id","goodreads_book_id"]],on="goodreads_book_id")
to_map = dict(zip(books.goodreads_book_id, books.book_id))
book_tags["book_id"] = book_tags.goodreads_book_id.map(to_map)

to_map[5]

18

### Задание 4.2
Далее нам необходимо оставить в наборе данных **book_tags** только те записи, теги для которых есть в данных **tags**.

Отфильтруйте данные таким образом, чтобы в наборе данных **book_tag**s остались только те строки, в которых находятся теги, информация о которых есть в наборе данных **tags**.

Сколько объектов осталось?

In [65]:
book_tags = book_tags[book_tags.tag_id.isin(tags.tag_id)]
book_tags.shape[0]

300738

Отлично, мы подготовили информацию о тегах книг — это будет метаинформацией для построения рекомендательной системы. Теперь нам необходимо подготовить данные о взаимодействии пользователей и книг. Для этого нам понадобится файл **ratings**.

Оба набора данных (и про взаимодействия, и про метаинформацию) необходимо преобразовать в разрежённые матрицы. Это можно сделать с помощью специальной функции из модуля `scipy`:

In [66]:
from scipy.sparse import csr_matrix

Осуществляем преобразование следующим образом:

In [67]:
# передаём в качестве аргументов в функцию выставленный рейтинг (это будут значения матрицы), 
# а также id пользователя и id книги (это будут индексы для строк и столбцов матрицы)
ratings_matrix = csr_matrix((ratings.rating,(ratings.user_id,ratings.book_id))) 

Теперь нам необходимо составить матрицу с метаданными. В качестве индексов будут выступать id книги и id тега, и если у этой книги есть рассматриваемый тег, то на пересечении соответствующих строки и столбца будет выставлена единица.

In [68]:
meta_matrix  = csr_matrix(([1]*len(book_tags),(book_tags.book_id,book_tags.tag_id))) 

### Задание 4.4
Давайте проверим, что всё получилось правильно.

Каково среднее арифметическое значений разрежённой матрицы с рейтингами? Ответ округлите до трёх знаков после точки-разделителя.

In [36]:
ans = ratings_matrix.mean()
round(ans,3)

0.007

Отлично, данные подготовлены — теперь настало время определить модель, которую мы будем использовать. Сделаем это следующим образом:

In [37]:
model = LightFM(loss='warp', #определяем функцию потерь
                random_state=13, #фиксируем случайное разбиение
                learning_rate=0.05, #темп обучения
                no_components=100) #размерность вектора для представления данных в модели

В качестве функции потерь мы выбрали значение 'warp', хотя, разумеется, это не единственный вариант. В модуле LightFM представлены следующие функции потерь:

* 'logistic' — логистическая функция. Полезна в случаях, когда есть как положительные, так и отрицательные взаимодействия, например 1 и -1.
* 'bpr' — байесовский персонализированный рейтинг. Можно применять, когда присутствуют только положительные взаимодействия.
* 'warp' — парный взвешенный приблизительный ранг. Используется, если необходимо повысить качество именно в верхней части списка рекомендаций.
* 'warp-kos' — модификация warp.

Разобьём данные на обучающую и тестовую выборки:

In [38]:
train,test = random_train_test_split(ratings_matrix, test_percentage=0.3, random_state=13)

Теперь обучим модель на наших данных о взаимодействии, также используя метаданные о книгах:
>Обратите внимание: из-за трудоёмкости вычислений обучение модели и оценка качества могут занимать вплоть до 15-20 минут (зависит от мощности компьютера). Не волнуйтесь, это нормальная ситуация.

In [39]:
%%time
model = model.fit(train, item_features = meta_matrix)

Wall time: 2min 57s


### Задание 4.5
Оцените качество полученной модели с помошью функции `precision_at_k`, передав в неё три аргумента: **модель**, **тестовые данные** и **обозначение метаданных** `(item_features = meta_matrix)`.

Выведите среднее арифметическое и округлите его до двух знаков после точки-разделителя.

In [41]:
%%time
prec_k = precision_at_k(model,
                        test,
                        item_features = meta_matrix)

0.02

In [43]:
ans = prec_k.mean()

0.017568793

In [ ]:
ans = 

## 5. Современные методы: глубокое обучение

Мы будем использовать модуль tensorflow, в котором реализовано много полезных методов для имплементации (внедрения) нейронных сетей.

Для начала импортируем из него функции, которые понадобятся нам для решения задачи:

In [1]:
from tensorflow.keras.layers import Input, Embedding, Flatten, Dot, Dense, Concatenate
from tensorflow.keras.models import Model

Мы будем использовать данные из предыдущего юнита, но лишь те, которые содержат информацию об оценках, выставленных книгам пользователями. Загрузим данные:

In [5]:
df = pd.read_csv('./data/ratings.csv')

### Задание 5.1
Разбейте данные на обучающую и тестовую выборки в отношении 4:1. В качестве значения параметра `random_state` возьмите число `42`.

Сколько объектов теперь находится в обучающей выборке?

In [6]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.2, random_state=42)
train.shape[0]

785404

### Задание 5.2
Сколько в наборе данных уникальных книг?

In [14]:
n_books = df.book_id.nunique()
n_books

10000

### Задание 5.3
Сколько в наборе данных уникальных пользователей?

In [20]:
n_users = df.user_id.nunique()
n_users

53424

В первую очередь нам необходимо создать эмбеддинги для книг и пользователей. Создаём эмбеддинги для книг:

In [15]:
book_input = Input(shape=[1], name="Book-Input")
book_embedding = Embedding(n_books+1, 5, name="Book-Embedding")(book_input)
book_vec = Flatten(name="Flatten-Books")(book_embedding)

Сначала мы задаём [размерность входного слоя][1]. После этого определяем размер эмбеддинга — в данном случае снижаем размерность до 5. Далее мы разворачиваем результат в массив с одним измерением с помощью слоя `Flatten()`.

[1]:: "В этом параметре максимальное значение всегда равно длинне вектора + 1"

Делаем то же самое для пользователей:

In [21]:
user_input = Input(shape=[1], name="User-Input")
user_embedding = Embedding(n_users+1, 5, name="User-Embedding")(user_input)
user_vec = Flatten(name="Flatten-Users")(user_embedding)

Теперь, когда мы создали представления как для книг, так и для пользователей, нам необходимо соединить их:

In [22]:
conc = Concatenate()([book_vec, user_vec])

Далее начинаем «собирать» нашу нейронную сеть из слоёв. `Dense` обозначает полносвязный слой. Также мы обозначаем для него [количество нейронов][1] и [данные][2], которые идут на вход.

[1]:: "В первом слое будет 128 нейронов, на втором - 32, на последенм(выходном) -1"
[2]:: "На первом слое принимаются данные от соединенных эмбендингов, на втором данные от первого слоя, на последнем - данные от второго"

In [23]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
out = Dense(1)(fc2)

Собираем модель — передаём входные данные для книг и пользователей, а также архитектуру нейронной сети:

In [24]:
model2 = Model([user_input, book_input], out)

Также нам необходимо задать алгоритм оптимизации и метрику, которую мы будем оптимизировать. В данном случае будем использовать метод [adam][1] и хорошо известную вам среднеквадратичную ошибку:

[1]:: "Это одна из вариаций градиентного спуска"

In [25]:
model2.compile(optimizer = 'adam',loss =  'mean_squared_error')

Теперь будем обучать нашу модель:

In [27]:
history = model2.fit([train.user_id, train.book_id], train.rating, epochs=5, verbose=1)

Epoch 1/5
24544/24544 [==============================] - 123s 5ms/step - loss: 0.7950
Epoch 2/5
24544/24544 [==============================] - 128s 5ms/step - loss: 0.6865
Epoch 3/5
24544/24544 [==============================] - 142s 6ms/step - loss: 0.6549
Epoch 4/5
24544/24544 [==============================] - 136s 6ms/step - loss: 0.6314
Epoch 5/5
24544/24544 [==============================] - 145s 6ms/step - loss: 0.6127


В параметр эпох передаём значение 5: у нас будет реализовано пять эпох — пять обучений нейронной сети. На каждой из эпох обновляются веса для минимизации ошибки.

Теперь можно оценить качество:

In [28]:
model2.evaluate([test.user_id, test.book_id], test.rating)

6136/6136 [==============================] - 15s 2ms/step - loss: 0.7091


0.7090549468994141

>Примечание. К сожалению, результаты этого алгоритма нельзя зафиксировать стандартным ramdom_state, к которому мы привыкли: применяемые методы не используют такой параметр. Поэтому мы опустим здесь сравнение результатов, однако посмотрим, как можно настроить нейронную сеть.

Обычно для улучшения качества модели каким-то образом модифицируют нейронную сеть: дополняют её, увеличивают время обучения. Добавим ещё один полносвязный слой с восемью нейронами после полносвязного слоя с 32 нейронами. Обучим нейронную сеть, реализовав десять эпох:

In [29]:
fc1 = Dense(128, activation='relu')(conc)
fc2 = Dense(32, activation='relu')(fc1)
fc3 = Dense(8, activation='relu')(fc2)
out = Dense(1)(fc3)

model2 = Model([user_input, book_input], out)
model2.compile('adam', 'mean_squared_error')
result = model2.fit([train.user_id, train.book_id], train.rating, epochs=10, verbose=1)
model2.evaluate([test.user_id, test.book_id], test.rating)

Epoch 1/10
24544/24544 [==============================] - 123s 5ms/step - loss: 0.6247
Epoch 2/10
24544/24544 [==============================] - 146s 6ms/step - loss: 0.5843
Epoch 3/10
24544/24544 [==============================] - 150s 6ms/step - loss: 0.5638
Epoch 4/10
24544/24544 [==============================] - 148s 6ms/step - loss: 0.5449
Epoch 5/10
24544/24544 [==============================] - 148s 6ms/step - loss: 0.5292
Epoch 6/10
24544/24544 [==============================] - 132s 5ms/step - loss: 0.5160
Epoch 7/10
24544/24544 [==============================] - 125s 5ms/step - loss: 0.5051
Epoch 8/10
24544/24544 [==============================] - 125s 5ms/step - loss: 0.4962
Epoch 9/10
6136/6136 [==============================] - 21s 3ms/step - loss: 0.7755


0.7754535675048828

## 6. Практика
Итак, вы познакомились с основными методами построения рекомендательных систем, и теперь настало время закрепить полученные знания на практике. В предыдущем модуле мы начали строить РС для сервиса чтения статей **CI&T DeskDrop**. В этом юните мы продолжим работу над ней.

In [33]:
df_articles = pd.read_csv("./data/shared_articles.csv")
df_interactions = pd.read_csv("./data/users_interactions.csv")
# приведем id к типу str - экономим память.
df_interactions.personId = df_interactions.personId.astype(str)
df_interactions.contentId = df_interactions.contentId.astype(str)
df_articles.contentId = df_articles.contentId.astype(str)
# Отфильтруем данные так, чтобы остались только объекты с типом события CONTENT SHARED.
shared_mask = df_articles.eventType == 'CONTENT SHARED'
df_shar_articles = df_articles[shared_mask]
# признак, который будет отражать числовой вес для взаимодействия со статьёй в соответствии 
# с приведёнными весами)
event_type = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}
df_interactions['weighted_eventType'] = df_interactions.eventType.map(event_type)
# оставим только тех пользователей, которые взаимодействовали хотя бы с пятью статьями
activity_mask = df_interactions.groupby('personId').contentId.nunique()>4
active_person = df_interactions.groupby('personId').contentId.count().index[activity_mask].to_list()
df_interactions = df_interactions[df_interactions.personId.isin(active_person)]
# логарифмирование суммы весов для взаимодействия пользователя с каждой конкретной статьёй
# также сохраним для каждой пары «пользователь — статья» значение времени последнего взаимодействия
def smooth_user_preference(x):
    return math.log(1+x, 2)
cum_weighted_event = df_interactions.groupby(['personId','contentId']).\
                        weighted_eventType.sum().apply(smooth_user_preference).reset_index()
last_interaction = df_interactions.groupby(['personId','contentId']).timestamp.last().\
                        reset_index().rename(columns={'timestamp':'last_timestamp'})
cum_weighted_event = cum_weighted_event.merge(last_interaction,on=['personId','contentId'])
# разделим данные на обучающую и тестовую выборки
split_timestamp = 1475519545
split_mask = cum_weighted_event.last_timestamp < split_timestamp

train = cum_weighted_event[split_mask].copy()
test = cum_weighted_event[~split_mask].copy()

Для начала необходимо построить матрицу, в которой по столбцам будут находиться `id статей`, по строкам — `id пользователей`, а на пересечениях строк и столбцов — оценка взаимодействия пользователя со статьёй. Если взаимодействия не было, в соответствующей ячейке должен стоять ноль.

In [34]:
interactions = pd.pivot_table(
    train,
    values='weighted_eventType',
    index='personId',
    columns='contentId').fillna(0)

### Задание 6.1
Найдите оценку взаимодействия пользователя с `ID` -1032019229384696495 со статьёй с `ID` 943818026930898372. Результат округлите до двух знаков после точки-разделителя.

>Примечание. Здесь и далее (пока не будет указано иное) необходимо работать с обучающей выборкой.

In [35]:
ans = interactions.loc["-1032019229384696495","943818026930898372"]
round(ans,2)

2.32

Теперь давайте попробуем применить memory-based-подход коллаборативной фильтрации.

>Примечание. Данных достаточно много, поэтому для увеличения скорости работы преобразуйте таблицу в массив numpy.

In [36]:
interactions_np = interactions.values

### Задание 6.2
Найдите среднее арифметическое всех чисел в получившемся массиве. Результат округлите до трёх знаков после точки-разделителя.

In [37]:
ans = interactions_np.mean()
round(ans,3)

0.017

>Перейдём к реализации коллаборативной фильтрации. Ранее мы делали это с помощью библиотеки surprise, однако это не всегда удобно, так как эта библиотека имеет ограниченное количество метрик для оценки качества и небольшой потенциал для более тонкой настройки алгоритма. Поэтому давайте попробуем реализовать алгоритмы коллаборативной фильтрации «с нуля». Такая практика применяется, если необходимо выстроить более сложную систему, чем могут предложить готовые модули. Кроме того, «ручная» реализация алгоритмов позволит лучше понять принцип их работы.

### Задание 6.3
Постройте матрицу схожести. Для этого вычислите все попарные коэффициенты корреляции для матрицы, полученной в предыдущем задании. Для каждой пары учитывайте только ненулевые значения (так как нулевые обозначают отсутствие взаимодействия и не интересуют нас). Выведите результат, полученный в ячейке с третьим индексом по строкам и сороковым — по столбцам.

In [38]:
%%time
size = len(interactions_np)
similarity_users = np.zeros((size, size))

for i in (range(size-1)):
    for j in range(i+1, size):
        mask_uv = (interactions_np[i] != 0) & (interactions_np[j] != 0)
        if mask_uv.sum() > 1:
            interactions_v = interactions_np[i, mask_uv]
            interactions_u = interactions_np[j, mask_uv]
            similarity_users[i,j] = np.corrcoef(interactions_v, interactions_u)[0, 1]
            similarity_users[j,i] = similarity_users[i,j]

ans = similarity_users[3,40]
round(ans,2)

C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_base.py:2642: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
C:\Users\Manych\anaconda3\lib\site-packages\numpy\lib\function_base.py:2643: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


Wall time: 32.7 s


-0.33

Теперь у нас есть матрицы схожести пользователей. Их можно использовать для построения рекомендаций. Чтобы это сделать, надо реализовать следующий алгоритм.

>Для каждого пользователя:   
>    
>1. Найти пользователей с похожестью больше 0.   
2. Для каждой статьи вычислить долю пользователей (среди выделенных на первом шаге), которые взаимодействовали со статьёй.   
3. Порекомендовать статьи с наибольшими долями со второго шага (среди тех, которые пользователь ещё не видел).

### Задание 6.4
Постройте рекомендательную систему по алгоритму, описанному выше. Найдите первую рекомендацию для строки 35 (если считать с нуля).

In [39]:
prediction_user_based = []
for i in range(len(similarity_users)):
    users_similarity_mask = similarity_users[i]>0
    
    
    aa = interactions_np[users_similarity_mask].sum(axis=1)[::-1]
    

contentId,-1006791494035379303,-1021685224930603833,-1022885988494278200,-1024046541613287684,-1033806831489252007,-1038011342017850,-1039912738963181810,-1046621686880462790,-1051830303851697653,-1055630159212837930,...,9217155070834564627,921770761777842242,9220445660318725468,9222265156747237864,943818026930898372,957332268361319692,966067567430037498,972258375127367383,980458131533897249,98528655405030624
personId,,,,,,,,,,,,,,,,,,,,,
-1007001694607905623,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
-1032019229384696495,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,3.0,0.0,0.0,0.0,2.321928,0.0,0.0,0.0,0.0,0.0
-108842214936804958,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,2.0,0.0,0.0,0.0
-1130272294246983140,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.000000,0.0,0.0,0.0,0.0,0.0
-1160159014793528221,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [42]:
popularity_similar = np.zeros_like(interactions_np)

In [76]:

# j = i+1

In [104]:
i = 4
users_similarity_mask = similarity_users[i]>0
interactions_np[users_similarity_mask].sum(axis=1)

array([943.15154013, 320.8590133 , 169.85288121,  86.6795587 ,
       176.77799869, 312.17977974, 358.08546689, 104.7695987 ])

In [105]:
users_similarity_mask = similarity_users[i]>0
popularity_similar[i,j] = (interactions_np[users_similarity_mask,j]>0).sum()
(interactions_np[users_similarity_mask,j]>0).sum(),popularity_similar[i,j]

(1, 1.0)

In [106]:
popularity_similar[i,j] = (interactions_np[users_similarity_mask,j]>0).sum()
aa = np.array(
    [(interactions_np[users_similarity_mask,x]>0).sum() 
    for x in range(interactions_np.shape[1])])[::-1]

In [107]:
aa#.argsort()[::-1]

array([943.15154013, 320.8590133 , 169.85288121,  86.6795587 ,
       176.77799869, 312.17977974, 358.08546689, 104.7695987 ])

In [ ]:
# sorted()

In [ ]:
# similar_users = interactions.index[similarity_mask]
# similar_users

## ==========================================

In [ ]:
https://more.tv/gosti_iz_proshlogo/1_sezon/9_seriya